In [ ]:
#hide
#test_flag_colab
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/spundsteif/spundsteif'
!pip3 install -e . -q
!pip3 install nbdev -q

#working directory: drive/MyDrive/spundsteif
%cd ..

%load_ext autoreload
%autoreload 2

Mounted at /content/drive
/content/drive/MyDrive/spundsteif/spundsteif
ERROR: File "setup.py" or "setup.cfg" not found. Directory cannot be installed in editable mode: /content/drive/MyDrive/spundsteif/spundsteif
     |████████████████████████████████| 46 kB 2.6 MB/s 
     |████████████████████████████████| 51 kB 309 kB/s 
     |████████████████████████████████| 56 kB 3.9 MB/s 
/content/drive/MyDrive/spundsteif


In [ ]:
# default_exp pressures

# Library pressures
> This library gives the earth pressure distribution along the sheet piles from the total stress given by the sensors and the piezometer data.  
> Some functions aiming a graphical check of measurements are also available.  
--- 
**Inputs:**  
+ `df_messung`: dataframe of the measurements from sensors
+ `df_piezo.csv`: csv file of the groudwater level
+ `df_excavation.csv`: csv file of the stage of excavation

**Outputs:**  
+ `df_pressures` : total, effective stresses and waterpressure along the sheetpiles.


## Importation des bibliothèques

In [ ]:
#export
import pandas as pd
import numpy
import io
import csv
from datetime import datetime

from google.colab import files

from copy import copy

from datetime import timedelta
from datetime import datetime

import plotly.express as px
import plotly.graph_objects as go 

import numpy as np

from spundsteif import sensors

## Test setup

### Test files

In [ ]:
PIEZO_FILE_NAME_TEST = './test/df_piezo_test.csv'
EXCAVATION_FILE_NAME_TEST = './test/df_excavation_stage_test.csv'

# Files needed to get df_measured_pressure
STRUCTURE_DATA = './test/1_structure_file.txt'

TESTS_FILES_NAMES = ['./test/1_data_test_01.ASC',
                     './test/1_data_test_02.txt',
                     './test/1_data_test_03.txt',
                     './test/1_data_test_04.ASC'
                     ]

TEST_HAND = './test/df_hand_test.csv'

### Measured pressure


In [ ]:
df_measured_pressure = sensors.get_data(list_files_names=TESTS_FILES_NAMES,
                                        structure_data=STRUCTURE_DATA, 
                                        file_name_df_hand=TEST_HAND)

df_measured_pressure = df_measured_pressure.loc[:,df_measured_pressure.columns.str.startswith('EDS')]

In [ ]:
df_measured_pressure.sample(10, random_state=0)

,EDS_A,EDS_B,EDS_C,EDS_D,EDS_E,EDS_F,EDS_G
2021-07-14 21:28:43.201370,-0.008392,0.443348,1.770932,-1.0,-0.013005,-6.332109,0.583440
2021-07-12 03:26:24.418810,-0.020542,-1.000000,-0.043438,-1.0,-0.013147,-6.065709,5.753562
2021-07-18 17:30:11.417100,0.011385,1.370416,2.557907,-1.0,0.001380,-6.130448,-3.851498
2021-07-15 19:29:03.205130,-0.003976,-1.000000,2.221444,-1.0,-0.013358,-6.334587,-1.914825
2021-07-08 19:25:08.005130,-0.015499,3.369134,-0.038738,-1.0,0.003474,-6.159601,-0.020800
2021-07-20 13:30:52.624630,0.017093,4.470687,2.422170,-1.0,0.000345,-6.302468,-4.388151
2021-08-18 00:00:00.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-07-19 13:30:29.620520,0.018375,4.841653,2.529231,-1.0,0.004594,-6.092389,-3.946954
2021-07-17 15:29:46.812660,0.010717,0.998889,2.490023,-1.0,0.006505,-6.300282,-3.930138
2021-07-08 14:25:04.204280,0.007363,1.635157,-0.019435,-1.0,0.025754,-6.187036,-0.024776


## Read files

### Read raw data

In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 01_sensors.ipynb.
Converted 02_inclinometers.ipynb.
Converted index.ipynb.
